In [11]:
import mysql.connector

AWS_mysql = mysql.connector.connect(
            host = "database-1.c1kkeqig4j9x.ap-northeast-2.rds.amazonaws.com",
            user = "root",
            password = "12345678",
            port = 3306,
            charset = "utf8mb4"
        )

In [13]:
# --- [3] 데이터베이스 목록 조회 ---
with AWS_mysql.cursor() as cursor:
    cursor.execute("SHOW DATABASES;")
    databases = cursor.fetchall()
    print("📂 데이터베이스 목록:")
    for db in databases:
        print("-", db[0])

📂 데이터베이스 목록:
- RECOVER_YOUR_DATA
- information_schema
- mysql
- performance_schema
- sys


In [20]:
AWS_mysql.close()

In [15]:
# --- [4] 특정 데이터베이스의 테이블 목록 조회 ---
db_name = "mysql"

with AWS_mysql.cursor() as cursor:
    cursor.execute(f"USE {db_name};")   # ✅ select_db 대신 USE 명령어 사용
    print(f"\n📂 데이터베이스 전환 완료: {db_name}")

    # --- [4] 테이블 목록 조회 ---
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print(f"\n📋 [{db_name}]의 테이블 목록:")
    for t in tables:
        print("-", t[0])


📂 데이터베이스 전환 완료: mysql

📋 [mysql]의 테이블 목록:
- columns_priv
- component
- db
- default_roles
- engine_cost
- func
- general_log
- global_grants
- gtid_executed
- help_category
- help_keyword
- help_relation
- help_topic
- innodb_index_stats
- innodb_table_stats
- password_history
- plugin
- procs_priv
- proxies_priv
- rds_configuration
- rds_global_status_history
- rds_global_status_history_old
- rds_heartbeat2
- rds_history
- rds_replication_status
- rds_reserved_users
- rds_upgrade_prechecks
- replication_asynchronous_connection_failover
- replication_asynchronous_connection_failover_managed
- replication_group_configuration_version
- replication_group_member_actions
- role_edges
- server_cost
- servers
- slave_master_info
- slave_relay_log_info
- slave_worker_info
- slow_log
- tables_priv
- time_zone
- time_zone_leap_second
- time_zone_name
- time_zone_transition
- time_zone_transition_type
- user


In [16]:
new_db_name = "FinalProject"

with AWS_mysql.cursor() as cursor:
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {new_db_name};")
    print(f"[✅ 데이터베이스 생성 완료] '{new_db_name}'")

# --- [3] 생성된 DB 확인 ---
with AWS_mysql.cursor() as cursor:
    cursor.execute("SHOW DATABASES;")
    databases = cursor.fetchall()
    print("\n📂 현재 존재하는 데이터베이스 목록:")
    for db in databases:
        print("-", db[0])

[✅ 데이터베이스 생성 완료] 'FinalProject'

📂 현재 존재하는 데이터베이스 목록:
- FinalProject
- RECOVER_YOUR_DATA
- information_schema
- mysql
- performance_schema
- sys


In [22]:
import mysql.connector

AWS_mysql = mysql.connector.connect(
            host = "database-1.c1kkeqig4j9x.ap-northeast-2.rds.amazonaws.com",
            user = "root",
            password = "12345678",
            port = 3306,
            database = "FinalProject",
            charset = "utf8mb4"
        )

In [23]:
with AWS_mysql.cursor() as cursor:
    # 1. 직원 관리 테이블
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            phone_suffix VARCHAR(4),
            hire_date DATE NOT NULL
        );
    """)

    # 2. 근태 테이블
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS attendance (
            id INT AUTO_INCREMENT PRIMARY KEY,
            employee_id INT NOT NULL,
            check_in DATETIME NOT NULL,
            check_out DATETIME,
            FOREIGN KEY (employee_id) REFERENCES employees(id)
        );
    """)

    # 3. 입출고 타임라인 테이블
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS inout_timeline (
            id INT AUTO_INCREMENT PRIMARY KEY,
            product_name VARCHAR(200) NOT NULL,
            inout_type ENUM('IN', 'OUT') NOT NULL,
            quantity INT NOT NULL,
            status ENUM('STORED', 'RETURNED') NOT NULL,
            timestamp DATETIME NOT NULL,
            confirmed_by INT,
            FOREIGN KEY (confirmed_by) REFERENCES employees(id)
        );
    """)

    # 4. 보관 테이블
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS storage (
            id INT AUTO_INCREMENT PRIMARY KEY,
            product_name VARCHAR(200) NOT NULL,
            location VARCHAR(20) NOT NULL,
            quantity INT NOT NULL,
            stored_at DATETIME NOT NULL
        );
    """)

    AWS_mysql.commit()
    print("[✅ 모든 테이블 생성 완료]")

[✅ 모든 테이블 생성 완료]


In [24]:
with AWS_mysql.cursor() as cursor:
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("📋 현재 데이터베이스의 테이블 목록:")
    for t in tables:
        print("-", t[0])

📋 현재 데이터베이스의 테이블 목록:
- attendance
- employees
- inout_timeline
- storage


In [25]:
with AWS_mysql.cursor() as cursor:
    cursor.execute("SHOW TABLES;")
    tables = [t[0] for t in cursor.fetchall()]
    
    for table in tables:
        print(f"\n📂 {table} 테이블 구조:")
        cursor.execute(f"DESCRIBE {table};")
        for col in cursor.fetchall():
            print(" ", col)


📂 attendance 테이블 구조:
  ('id', 'int', 'NO', 'PRI', None, 'auto_increment')
  ('employee_id', 'int', 'NO', 'MUL', None, '')
  ('check_in', 'datetime', 'NO', '', None, '')
  ('check_out', 'datetime', 'YES', '', None, '')

📂 employees 테이블 구조:
  ('id', 'int', 'NO', 'PRI', None, 'auto_increment')
  ('name', 'varchar(100)', 'NO', '', None, '')
  ('phone_suffix', 'varchar(4)', 'YES', '', None, '')
  ('hire_date', 'date', 'NO', '', None, '')

📂 inout_timeline 테이블 구조:
  ('id', 'int', 'NO', 'PRI', None, 'auto_increment')
  ('product_name', 'varchar(200)', 'NO', '', None, '')
  ('inout_type', "enum('IN','OUT')", 'NO', '', None, '')
  ('quantity', 'int', 'NO', '', None, '')
  ('status', "enum('STORED','RETURNED')", 'NO', '', None, '')
  ('timestamp', 'datetime', 'NO', '', None, '')
  ('confirmed_by', 'int', 'YES', 'MUL', None, '')

📂 storage 테이블 구조:
  ('id', 'int', 'NO', 'PRI', None, 'auto_increment')
  ('product_name', 'varchar(200)', 'NO', '', None, '')
  ('location', 'varchar(20)', 'NO', '', No

In [26]:
AWS_mysql.close()